## 🛠️ **ETL (Extract, Transform, Load)**



En este ipynb vamos a Extraer los archivos y cargarlos en parquet. Luego en otro archivo haremos las transformaciones.

Tambien vamos a estar utilizando la funcion personalizada personalizada `data_type_check` invocada desde `data_utils.py` para dejar un vistazo raápido del dataframe y  poder observar:
- Variables categóricas
- Variables numéricas
- Dimensiones del dataframe
- Nulos
- Tipos de datos
- Informacion acerca de los datos faltantes o nulos de cada columna    


####  **Importamos las librerías que vamos a usar**


In [1]:
import gc
import os
import warnings

import pandas as pd
from data_utils import data_type_check, data_type_check_pkl

warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import seaborn as sns

___

## Dataset Yelp

### business.pkl

In [6]:
business = pd.read_parquet('../0_Dataset/Yelp/')
business.info()
business.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
15195,u_0e1X9whtdIBURmiQCv_A,Burrito Boarder,17 3rd St N,Saint Petersburg,TN,33701,27.771694,-82.636936,3.0,190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99735,QTmwmIb8AHhyQBB2Q19xsg,Tommy's Express® Car Wash,1240 Missouri Ave N,Largo,DE,33770,27.928671,-82.78711,3.5,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Tenemos columnas duplicadas, procedemos a quitarlas.


In [7]:
business = business.loc[:, ~business.columns.duplicated()]
business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
dtypes: object(14)
memory usage: 17.2+ MB


#### 🔁 **TRANSFORM**

Verificamos los nulos y duplicados de categories

In [8]:
# Cuenta los duplicados y nulos en la columna 'categories' antes de cualquier eliminación
duplicados_inicial = business['categories'].duplicated().sum()
nulos_inicial = business['categories'].isna().sum()

print("Antes: Cantidad de duplicados en 'categories':", duplicados_inicial)
print("Antes: Cantidad de NaNs en 'categories':", nulos_inicial)

# Elimina las filas con valores NaN en la columna 'categories'
business = business.dropna(subset=['categories'])

# Cuenta los duplicados y nulos en la columna 'categories' después de eliminar NaNs
duplicados_despues_nulos = business['categories'].duplicated().sum()
nulos_despues_nulos = business['categories'].isna().sum()

print("Después: Cantidad de NaNs en 'categories' ", nulos_despues_nulos)


'''
________________________________________________________________
FUTURO: Descomentar o borrar
Elimina las filas duplicadas basadas en la columna 'categories'
business = business.drop_duplicates(subset=['categories'])  
_________________________________________________________________  
'''

# Cuenta los duplicados en la columna 'categories' después de eliminar duplicados
duplicados_final = business['categories'].duplicated().sum()

print("Después: Cantidad de duplicados en 'categories'", duplicados_final)


Antes: Cantidad de duplicados en 'categories': 67185
Antes: Cantidad de NaNs en 'categories': 103
Después: Cantidad de NaNs en 'categories'  0
Después: Cantidad de duplicados en 'categories' 67083


Dado que diferentes negocios pueden compartir las mismas 'categories', no borramos los  67083 duplicados 


In [9]:
data_type_check(business)


 Resumen del dataframe:

Dimensiones:  (150243, 14)
         columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0    business_id      100.00     0.00            0    object
1           name      100.00     0.00            0    object
2        address      100.00     0.00            0    object
3           city      100.00     0.00            0    object
4          state      100.00     0.00            3    object
5    postal_code      100.00     0.00            0    object
6       latitude      100.00     0.00            0    object
7      longitude      100.00     0.00            0    object
8          stars      100.00     0.00            0    object
9   review_count      100.00     0.00            0    object
10       is_open      100.00     0.00            0    object
11    attributes       90.92     9.08        13642    object
12    categories      100.00     0.00            0    object
13         hours       84.61    15.39        23120    object


#### **📤 LOAD**

In [10]:
#guardar en parquet
business.to_parquet("../0_Dataset/Yelp/business.parquet", engine="pyarrow")

### review.json


📤 LOAD

### checkin.json

Tamaño del archivo Parquet reducido: 87.89 MB
El archivo es menor de 90 MB. Podrías aumentar la fracción.


### tip.json

### user.parquet


Muestra guardada en ../0_Dataset/Yelp/user_reducido.parquet


___

___
## Dataset Gogle


#### **📂Procesamiento del 1er archivo: `Google Maps/metadata-sitios/review-Florida-`**

### Reviews Florida

#### **📤 LOAD**

### Metadata-sitios

🌟 La "metadata" incluye la información de diversos establecimientos en Google Maps.

📂 Esta información está dividida en 11 archivos JSON, organizados en 3 carpetas para facilitar el procesamiento y almacenamiento.

In [ ]:
data_type_check(df_rev_FL)

📤 LOAD

In [ ]:
#  Posibles soluciones: Exportar multiples parquet de metadata y unirlos luego 
 